使用R语言学习G估算技术。

In [22]:
library(tidyverse)
library(broom)

In [23]:
nhefs <- read_csv("./data/nhefs.csv")

head(nhefs)

Rows: 1629 Columns: 64
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (64): seqn, qsmk, death, yrdth, modth, dadth, sbp, dbp, sex, age, race, ...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


seqn,qsmk,death,yrdth,modth,dadth,sbp,dbp,sex,age,...,birthcontrol,pregnancies,cholesterol,hightax82,price71,price82,tax71,tax82,price71_82,tax71_82
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
233,0,0,NA,NA,NA,175,96,0,42,...,2,NA,197,0,2.183594,1.739990,1.1022949,0.4619751,0.44378662,0.6403809
235,0,0,NA,NA,NA,123,80,0,36,...,2,NA,301,0,2.346680,1.797363,1.3649902,0.5718994,0.54931641,0.7929688
244,0,0,NA,NA,NA,115,75,1,56,...,0,2,157,0,1.569580,1.513428,0.5512695,0.2309875,0.05619812,0.3202515
245,0,1,85,2,14,148,78,0,68,...,2,NA,174,0,1.506592,1.451904,0.5249023,0.2199707,0.05479431,0.3049927
252,0,0,NA,NA,NA,118,77,0,40,...,2,NA,216,0,2.346680,1.797363,1.3649902,0.5718994,0.54931641,0.7929688
257,0,0,NA,NA,NA,141,83,1,43,...,0,1,212,1,2.209961,2.025879,1.1547852,0.7479248,0.18408203,0.4069824


In [24]:
nhefs$cens <- ifelse(is.na(nhefs$wt82), 1, 0)

In [25]:
library(Hmisc)
describe(nhefs$wt82_71)

nhefs$wt82_71 
       n  missing distinct     Info     Mean  pMedian      Gmd      .05 
    1566       63     1510        1    2.638    2.607    8.337   -9.752 
     .10      .25      .50      .75      .90      .95 
  -6.292   -1.478    2.604    6.690   11.117   14.739 

lowest : -41.2805 -30.5019 -30.0501 -29.0258 -25.9706
highest: 34.0178  36.9693  37.6505  47.5113  48.5384 

In [26]:
# 估计逆概率权重，G估算不能控制选择偏倚
# 估计选择模型
cw.denom <- glm(
  cens==0 ~ qsmk + sex + race + age + I(age^2) 
  + as.factor(education) + smokeintensity + I(smokeintensity^2) 
  + smokeyrs + I(smokeyrs^2) + as.factor(exercise)
  + as.factor(active) + wt71 + I(wt71^2),
  data = nhefs,
  family = binomial("logit")
)

tidy(cw.denom)

# 计算逆概率权重
nhefs$pd.c <- predict(cw.denom, nhefs, type="response")
nhefs$wc <- ifelse(nhefs$cens == 0, 1 / nhefs$pd.c, NA)  # observations with cens=1 only contribute to censoring models

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-4.0144660794,2.5761057592,-1.55834677,0.119151068
qsmk,-0.5168674288,0.2877161756,-1.79644898,0.072423141
sex,-0.0573130913,0.3302775370,-0.17353009,0.862234777
race,0.0122714864,0.4524887466,0.02711998,0.978364038
age,0.2697292970,0.1174646830,2.29625867,0.021661096
I(age^2),-0.0028836512,0.0011135046,-2.58970754,0.009605750
as.factor(education)2,0.4407884451,0.4193992939,1.05099949,0.293258823
as.factor(education)3,0.1646881298,0.3705471430,0.44444582,0.656720290
as.factor(education)4,-0.1384469859,0.5697969170,-0.24297602,0.808023976


In [45]:
library("geepack")

nhefs$psi <- 3.446
nhefs$Hpsi <- nhefs$wt82_71 - nhefs$psi*nhefs$qsmk

fit <- geeglm(qsmk ~ sex + race + age + I(age*age) + as.factor(education)
           + smokeintensity + I(smokeintensity*smokeintensity) + smokeyrs
           + I(smokeyrs*smokeyrs) + as.factor(exercise) + as.factor(active)
           + wt71 + I(wt71*wt71) + Hpsi, family=binomial, data=nhefs,
           weights=wc, id=seqn, corstr="independence")

tidy(fit)

Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-2.403e+00,1.3290542,3.269e+00,7.060e-02
sex,-5.137e-01,0.1535534,1.119e+01,8.210e-04
race,-8.609e-01,0.2098745,1.683e+01,4.097e-05
age,1.152e-01,0.0501955,5.263e+00,2.178e-02
I(age * age),-7.593e-04,0.0005296,2.056e+00,1.516e-01
as.factor(education)2,-2.894e-02,0.1963924,2.171e-02,8.829e-01
as.factor(education)3,8.771e-02,0.1725907,2.582e-01,6.113e-01
as.factor(education)4,6.637e-02,0.2697603,6.054e-02,8.056e-01
as.factor(education)5,4.711e-01,0.2246983,4.395e+00,3.604e-02


In [47]:
# 单个可能值测试
nhefs$psi <- 3.446
nhefs$Hpsi <- nhefs$wt82_71 - nhefs$psi*nhefs$qsmk

# 除去权重缺失的观测
nhefs <- nhefs %>% filter(cens == 0)

library(survey)

fit <- svyglm(
  qsmk ~ sex + race + age + I(age * age) + as.factor(education)
  + smokeintensity + I(smokeintensity * smokeintensity) + smokeyrs
  + I(smokeyrs * smokeyrs) + as.factor(exercise) + as.factor(active)
  + wt71 + I(wt71 * wt71) + Hpsi,
  design = svydesign(id = ~1, weights = ~wc, data = nhefs),
  family = binomial("logit")
)

tidy(fit)

Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-2.403e+00,1.3294787,-1.8074406,7.089e-02
sex,-5.137e-01,0.1536024,-3.3445594,8.439e-04
race,-8.609e-01,0.2099415,-4.1006236,4.335e-05
age,1.152e-01,0.0502116,2.2934732,2.195e-02
I(age * age),-7.593e-04,0.0005297,-1.4333790,1.520e-01
as.factor(education)2,-2.894e-02,0.1964551,-0.1472989,8.829e-01
as.factor(education)3,8.771e-02,0.1726458,0.5080150,6.115e-01
as.factor(education)4,6.637e-02,0.2698465,0.2459691,8.057e-01
as.factor(education)5,4.711e-01,0.2247701,2.0958545,3.626e-02


In [56]:
fit$coefficients["Hpsi"]

Hpsi 
-1.903e-06

In [57]:
result <- tidy(fit)

In [64]:
# 提取Hpsi的系数
Hpsi_coef <- result %>% filter(term == "Hpsi") %>% select(estimate) %>% pull()

Hpsi_coef

[1] -1.903e-06

In [67]:
grid <- seq(from = 2, to = 5, by = 0.1)
j = 0
Hpsi.coefs <- cbind(rep(NA,length(grid)), rep(NA, length(grid)))
colnames(Hpsi.coefs) <- c("Estimate", "p-value")

for (i in grid){
  psi = i
  j = j+1
  nhefs$Hpsi <- nhefs$wt82_71 - psi * nhefs$qsmk 

  gest.fit <- svyglm(
    qsmk ~ sex + race + age + I(age*age) + as.factor(education)
    + smokeintensity + I(smokeintensity*smokeintensity) + smokeyrs
    + I(smokeyrs*smokeyrs) + as.factor(exercise) + as.factor(active)
    + wt71 + I(wt71*wt71) + Hpsi,
    design = svydesign(id = ~1, weights = ~wc, data = nhefs),
    family = binomial("logit")
  )

  # 计算估计值和p值
  Hpsi.coefs[j,1] <- tidy(gest.fit)$estimate[tidy(gest.fit)$term == "Hpsi"]
  Hpsi.coefs[j,2] <- tidy(gest.fit)$p.value[tidy(gest.fit)$term == "Hpsi"]
}

# 合并Grid和Hpsi系数
Hpsi.coefs <- cbind(grid, Hpsi.coefs)
colnames(Hpsi.coefs) <- c("psi", "Estimate", "p-value")
Hpsi.coefs

Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$initialize):
"non-integer #successes in a binomial glm!"
Warning message in eval(family$i

psi,Estimate,p-value
2.0,0.0267219,0.001812
2.1,0.0248946,0.003642
2.2,0.0230655,0.007057
2.3,0.0212344,0.013160
2.4,0.0194009,0.023599
2.5,0.0175647,0.040663
2.6,0.0157254,0.067294
2.7,0.0138827,0.106942
2.8,0.0120362,0.163212
2.9,0.0101857,0.239310
